In [ ]:
#imports here
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.support.wait import WebDriverWait

import re
import time
import os

import schedule

### Launch Selenium browser

In [ ]:
service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://www.facebook.com/")

### Open livestream link

In [ ]:
#livestream link
url_stream = "https://www.facebook.com/watch/live/?ref=watch_permalink&v=1090523771497591"
driver.get(url_stream)

In [ ]:
views_data_raw = [] #I usually comment this out immediately after running so I dont run the risk of resetting the views list container

### Setup directory for event

In [ ]:
event_name = 'LK_Pasay'
count=0

In [ ]:
#must have figs/ and data/ in home dir
os.makedirs('figs/'+event_name, exist_ok=True)

In [ ]:
#sample screenshot
current_dt = pd.Timestamp.now(tz='Asia/Manila').strftime("%Y-%m-%dT%H:%M:%S")
"figs/"+event_name+"/ss_"+event_name+'_'+current_dt.replace(":","").replace(" ","").replace("-","")+".png"

### Define job

In [ ]:
def job():
    current_dt = pd.Timestamp.now(tz='Asia/Manila').strftime("%Y-%m-%dT%H:%M:%S")
    driver.get(url_stream)
    
    #browser waits for 10 seconds or until livestream video appears 
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[data-pagelet='WatchPermalinkVideo']")))
    
    # encode browser rendered html
    search_result_source = driver.page_source.encode('utf-8').decode('ascii', 'ignore')
    # find anchor string 'people currently wathcing'
    views_idx = search_result_source.find('people currently watching')
    # get characters around anchor, split text and remove whitespace 
    num_views_str = search_result_source[views_idx-10:views_idx].split('"')[1].strip()
    # append to container list
    views_data_raw.append((current_dt,num_views_str))
    
    # printout views data 
    print(current_dt+': '+num_views_str+ ' views')
    
    # Take screenshots and save data as csv
    # less frequent screenshots at beginning of program
    mins_interval_ss = 1 if current_dt.split("T")[1]>="18:00:00" else 2
    
    #do these actions at specific minutes
    if current_dt.split("T")[1].split(":")[1] in ['%02d' % c for c in np.arange(0,60,mins_interval_ss)]:
        #build ss filename and save 
        fig_fn = "figs/"+event_name+"/ss_"+event_name+'_'+current_dt.replace(":","").replace(" ","").replace("-","")+".png"
        driver.get_screenshot_as_file(fig_fn)
        
        #create dataframe from container list and save as csv
        df = pd.DataFrame(views_data_raw, columns=['dt','views_K'])
        df['views_K'] = df['views_K'].apply(lambda x: float(x.replace("K","")))
        df.to_csv('data/'+event_name+"_livestream_views.csv", index=False)

In [ ]:
job() #test one job run

### Schedule job

In [ ]:
#clear sched to remove existing jobs
schedule.clear()
#change until date to a time when youd expect the livestream to end (plus allowance)
current_date = current_dt.split("T")[0]
schedule.every(1).minutes.at(":15").until(current_date+" 23:55").do(job)
while True:
    try:
        schedule.run_pending()
        time.sleep(1)    
    except Exception as e:
        print('ERROR here')
        print(e)
        continue